In [1]:
import praw
import time

#### Due to the nature of scraping this data(apparently the current implementation of rate_limiting is bugged), I chunked up the data in multiple csvs.

- reddit_questions.csv - 4615 rows - Quick proof of concept. Dirty Data. 

        -TrueAskReddit
        -AskHistorians
        -AskScienceFiction
        -NoStupidQuestions
        -relationship_advice
        -AskProgramming
        -AskAcademia
        -AskPhilosophy
        -AskMen
        -AskWomen
        -AskStatistics
- reddit_questions2.csv - 2498 rows - Improved selection and cleaning

        -TrueAskReddit
        -AskHistorians
        -AskScienceFiction
        -NoStupidQuestions
        -relationship_advice
        -AskProgramming
- reddit_questions3.csv - 1136 rows - Even More Improved selection and cleaning

        -AskAcademia
        -AskPhilosophy
        -Ask_Politics
- reddit_questions4.csv - 4168 rows  - Even More Improved selection and cleaning

        -AskGames
        -AskStatistics
        -answers
        -AskAnthropology
        -relationship_advice
        -AskSocialScience
        -AskScience
        -AskWomen
        -AskMen
        -AskScience
        -AskReddit
- reddit_questions5.csv - 2188 rows - Even More improved selection and cleaning

        -TrueAskReddit
        -AskHistorians
        -NoStupidQuestions
        -AskProgramming
        


In [2]:
reddit = praw.Reddit(ratelimit_seconds=600) 

MissingRequiredAttributeException: Required configuration setting 'client_id' missing. 
This setting can be provided in a praw.ini file, as a keyword argument to the Reddit class constructor, or as an environment variable.

In [5]:
subreddits = ['NoStupidQuestions',
              'AskHistorians',
              'TrueAskReddit',
              'AskProgramming']

The following is the crux of the reddit scraping. I go through each post and check for valid question-answer pairs and put them in the subreddit_dict in the following structure:

{'subreddit name': \[(Q1,A1),(Q2,A2),...],
 'subreddit name 2': \[(Q1,A1),(Q2,A2),...]'}
 


* I'd like to thank Vijay for giving me the insight to check for 'stickied posts' to filter out moderator answers. Originally, I was just checking to see if 'Welcome' or 'welcome' is in the comment. Vijay also gave me the suggestion to use the ratelimit_seconds argument in the Reddit instantiation to help with my TooManyRequests errors *

* I'd like thank Zubeir for giving me the insight on using a sleep mechanism to slow everything down*

* I'd also like to thank Eric for his breakfast hour session. I downloaded that video, and it walked me through this whole process and significantly cut my time to make this*

In [6]:

subreddit_dict = {}

# Loop through subreddits
for subreddit in subreddits:
    
    # storing questions in the following listas tuples:(Question, ANswer)
    qa_pairs = []
    
    # do a special case for r/relationship_advice
    if subreddit == 'relationship_advice':
        top_posts = reddit.subreddit(subreddit).top(limit=10000)
    else:
        top_posts = reddit.subreddit(subreddit).top(limit=1000)
    # Loop through top posts in the subreddit    
    for post in top_posts:
        
        # Sleep for .25 seconds between checking posts. I'm tired of getting kicked out
        time.sleep(.25)
        
        # Make sure the post has a title and comments 
        if post.title and post.comments:
            
            title = post.title
            # Make sure it's a question and try to not capture the questions directed explicitly at humans and no mentions of reddit
            if (title[-1] == '?') and ('you' not in title) and ('You' not in title) and ('eddit' not in title):
                # make the post title the question
                question = title
                #the first comment shuld be the most upvoted
                top_comment = post.comments[0] 
                # Check if the top comment is a valid answer (not pinned or removed or deleted)
                if (not top_comment.stickied) and (top_comment.body != '[removed]') and (top_comment.body != '[deleted]'):
                    
                    answer = top_comment.body
                    qa_pairs.append((question, answer))
                    #I used the following for debugging 
                    # print(f'Subreddit: {subreddit}')
                    # print('-')
                    # print(f'Question: {question}')
                    # print('-')
                    # print(f'Answer: {answer[:100]}')
                    # print('!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                    
                    time.sleep(1)


                      
        #  Once we get 500 questions, that's all we need ( i need to add this, because it's possible that we check 1000 posts)
        # if len(qa_pairs) == 500: 
        #     break
    
    # add qa pairs list as a value to the subreddit key
    subreddit_dict[subreddit] = qa_pairs 
    print(f'sleeping for 3 seconds. {len(qa_pairs)} qa pairs from  r/{subreddit}')
    time.sleep(3)

sleeping for 3 seconds. 618 qa pairs from  r/NoStupidQuestions
sleeping for 3 seconds. 482 qa pairs from  r/AskHistorians
sleeping for 3 seconds. 569 qa pairs from  r/TrueAskReddit
sleeping for 3 seconds. 519 qa pairs from  r/AskProgramming


Next step is to save all this information in a csv file. 

I use pandas, and unpack and repack my subreddit_dict into a better more easier object so that I can put it in the df that I create. 

There's definitely a better way to do this. I can perhaps make the df in the above loop itself. Or make the dict in such a way that i dont need to unpack/repack into a different object and therefore feed it directly into DataFrame constructor.

This was an easy way for me to conceptualize and execute quickly. And it works how I want it to.

In [15]:
import pandas as pd # My argument for not putting all my imports up top: I wanted to show my thought process from start to finish. 

In [8]:
df = pd.DataFrame()

In [9]:
cleanedL_firstfive = []
for subreddit in subreddit_dict.keys():
    for t in subreddit_dict[subreddit]:
        cleanedL_firstfive.append((subreddit,t[0],t[1]))   

In [10]:
len(cleanedL_firstfive)

2188

In [11]:
df['Subreddit'] = [t[0] for t in cleanedL_firstfive]
df['Questions'] = [t[1] for t in cleanedL_firstfive]
df['Human_Answer'] = [t[2] for t in cleanedL_firstfive]

In [12]:
df.head()

,Subreddit,Questions,Human_Answer
0,NoStupidQuestions,"Why is it ""price gouging"" when people resell s...",Who said either of those was ok?
1,NoStupidQuestions,Does anyone else stay up at night because they...,Nearly every night. I don’t want life to just...
2,NoStupidQuestions,Does anybody else hate how the Internet now fe...,Stumbleupon was awesome a decade ago.
3,NoStupidQuestions,Does anyone else stay up at night because that...,Yep
4,NoStupidQuestions,Is it weird that my 35 year old friend is wait...,"Meanwhile, in r/teenagers, there’s a 17 year o..."


In [13]:
df.to_csv('data/reddit_questions5.csv')